[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_lora.ipynb)

In [ ]:
!git clone -b v0.4.4 https://github.com/camenduru/sd-scripts
!sed -i -e 's/requests==2.28.2/# requests==2.28.2/' /content/sd-scripts/requirements.txt
%cd /content/sd-scripts
!pip install -r requirements.txt
!pip install --pre xformers
!pip install --pre triton

In [ ]:
!pip --pre xformers!python -u finetune/tag_images_by_wd14_tagger.py /content/drive/MyDrive/AI/training/parkminyoung --repo_id SmilingWolf/wd-v1-4-convnext-tagger-v2 --model_dir wd14_tagger_model --thresh 0.35 --batch_size 1 --caption_extension .txt

In [ ]:
!python -u finetune/merge_dd_tags_to_metadata.py /content/drive/MyDrive/AI/training/parkminyoung /content/drive/MyDrive/AI/training/parkminyoung/parkminyoung.json --caption_extension .txt

In [ ]:
!python -u finetune/prepare_buckets_latents.py /content/drive/MyDrive/AI/training/parkminyoung /content/drive/MyDrive/AI/training/parkminyoung/parkminyoung.json /content/drive/MyDrive/AI/training/parkminyoung/parkminyoung-lat.json JosephusCheung/ACertainty --batch_size 1 --max_resolution 512,512 --min_bucket_reso 256 --max_bucket_reso 1024 --bucket_reso_steps 64 --mixed_precision no

In [ ]:
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("JosephusCheung/ACertainty", safety_checker=None).to("cuda")
pipe.enable_xformers_memory_efficient_attention()
image = pipe("duck", num_inference_steps=25, guidance_scale=7).images[0]
display(image)

In [ ]:
!wget https://huggingface.co/ckpt/ACertainty/resolve/main/ACertainty.ckpt

In [ ]:
!python -u train_network.py --pretrained_model_name_or_path JosephusCheung/ACertainty --train_data_dir /content/drive/MyDrive/AI/training/parkminyoung --in_json /content/drive/MyDrive/AI/training/parkminyoung/parkminyoung-lat.json --output_dir /content/trained --xformers --max_train_steps 1600 --use_8bit_adam --network_module networks.lora

In [ ]:
!rm /content/drive/MyDrive/AI/training/parkminyoung/*.npz
!rm /content/drive/MyDrive/AI/training/parkminyoung/parkminyoung-lat.json